In [1]:
import altair as alt

cars = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/cars.json'
movies = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/movies.json'
sp500 = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/sp500.csv'
stocks = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/stocks.csv'
flights = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/flights-5k.json'

# 6.6. Detalhes sob demanda (Giovani)

Uma vez que identificamos pontos de interesse em uma visualização, muitas vezes queremos saber mais sobre eles. _Detalhes sob demanda_ refere-se à capacidade de consultar interativamente mais informações sobre valores selecionados. As _tooltips_ são um meio útil para fornecer esses detalhes sob demanda. No entanto, as tooltips geralmente exibem informações apenas para um único ponto de dados por vez. Como podemos mostrar mais?

O gráfico de dispersão das avaliações de filmes inclui vários outliers potencialmente interessantes, onde as classificações do Rotten Tomatoes e do IMDB divergem. Vamos criar um gráfico que nos permita selecionar interativamente pontos e exibir seus rótulos. Para ativar a consulta de filtro tanto na interação de passar o mouse quanto no clique, usaremos o [operador de composição do Altair](https://altair-viz.github.io/user_guide/interactions.html#composing-multiple-selections) `|` ("ou").

_Passe o mouse sobre os pontos no gráfico de dispersão abaixo para ver um destaque e um rótulo de título. Pressione Shift e clique nos pontos para tornar as anotações mais persistentes e visualizar vários rótulos ao mesmo tempo. Quais filmes são amados pelos críticos do Rotten Tomatoes, mas não pelo público geral no IMDB (ou vice-versa)? Veja se consegue encontrar possíveis erros, como dois filmes diferentes com o mesmo nome que foram acidentalmente combinados!_

In [2]:
hover = alt.selection_single(
    on='mouseover',  # select on mouseover
    nearest=True,    # select nearest point to mouse cursor
    empty='none'     # empty selection should match nothing
)

click = alt.selection_multi(
    empty='none' # empty selection matches no points
)

# scatter plot encodings shared by all marks
plot = alt.Chart().mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q'
)

# shared base for new layers
base = plot.transform_filter(
    hover | click # filter to points in either selection
)

# layer scatter plot points, halo annotations, and title labels
alt.layer(
    plot.add_selection(hover).add_selection(click),
    base.mark_point(size=100, stroke='firebrick', strokeWidth=1),
    base.mark_text(dx=4, dy=-8, align='right', stroke='white', strokeWidth=2).encode(text='Title:N'),
    base.mark_text(dx=4, dy=-8, align='right').encode(text='Title:N'),
    data=movies
).properties(
    width=600,
    height=450
)

<ipython-input-2-7e4997d76828>:1: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  hover = alt.selection_single(
<ipython-input-2-7e4997d76828>:7: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  click = alt.selection_multi(
<ipython-input-2-7e4997d76828>:24: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  plot.add_selection(hover).add_selection(click),


alt.LayerChart(...)


O exemplo acima adiciona três novas camadas ao gráfico de dispersão: uma  anotação circular, texto branco para fornecer um fundo legível, e texto preto exibindo o título do filme. Além disso, esse exemplo usa duas seleções em conjunto:

1. Uma seleção única (`hover`), que inclui `nearest=True` para selecionar automaticamente o ponto de dados mais próximo conforme o mouse se move.
2. Uma seleção múltipla (`click`), que permite criar seleções persistentes usando Shift + clique.



Ambas as seleções incluem a configuração `empty='none'` para indicar que nenhum ponto deve ser incluído se a seleção estiver vazia. Essas seleções são então combinadas em um único predicado de filtro — o operador lógico _ou_ entre `hover` e `click` — para incluir os pontos pertencentes a _qualquer uma_ das seleções. Esse predicado é usado para filtrar as novas camadas e exibir anotações e rótulos apenas para os pontos selecionados.

Usando seleções e camadas, podemos criar diversos designs para exibir detalhes sob demanda! Por exemplo, aqui está uma série temporal em escala logarítmica com os preços de ações de tecnologia, anotada com uma linha-guia e rótulos para a data mais próxima do cursor do mouse:

In [3]:
# select a point for which to provide details-on-demand
label = alt.selection_single(
    encodings=['x'], # limit selection to x-axis value
    on='mouseover',  # select on mouseover events
    nearest=True,    # select data point nearest the cursor
    empty='none'     # empty selection includes no data points
)

# define our base line chart of stock prices
base = alt.Chart().mark_line().encode(
    alt.X('date:T'),
    alt.Y('price:Q', scale=alt.Scale(type='log')),
    alt.Color('symbol:N')
)

alt.layer(
    base, # base line chart

    # add a rule mark to serve as a guide line
    alt.Chart().mark_rule(color='#aaa').encode(
        x='date:T'
    ).transform_filter(label),

    # add circle marks for selected time points, hide unselected points
    base.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
    ).add_selection(label),

    # add white stroked text to provide a legible background for labels
    base.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=2).encode(
        text='price:Q'
    ).transform_filter(label),

    # add text labels for stock prices
    base.mark_text(align='left', dx=5, dy=-5).encode(
        text='price:Q'
    ).transform_filter(label),

    data=stocks
).properties(
    width=700,
    height=400
)

<ipython-input-3-84fbeb2ff33e>:2: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  label = alt.selection_single(
<ipython-input-3-84fbeb2ff33e>:27: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).add_selection(label),


alt.LayerChart(...)

_Colocando em prática o que aprendemos até agora: você consegue modificar o gráfico de dispersão de filmes acima (aquele com consulta dinâmica ao longo dos anos) para incluir uma marca de regra que exiba a média das avaliações do IMDB (ou Rotten Tomatoes) dentro do intervalo de anos selecionado?_